<center> <h1 style="background-color:seagreen; color:white" >Construindo Pipelines e Ajuste com Train-Validation Split com Spark</h1></center>  

**Ajuste de Hiper Parâmetros:**

> + Um classificador pode ter diversos hiper parâmetros
> + A configuração dos hiper parâmetros as vezes não é intuitiva e nem “cientifica”
> + A combinação de muitos hiper parâmetros pode ser gigantesca
> + Inviável testar manualmente todas as combinações

> + Spark facilita o teste de diferentes hiper parâmetros
> + Você pode testar diferentes configurações para todo o Pipeline
> + ParamGridBuilder: você configura quais parâmetros e quais domínios quer testar
> + Utiliza Cross-Validation ou Train-Validation Split


<center> <h2 style="background-color:DarkKhaki; color:white" >Inicializando o PySpark</h2></center> 

In [1]:
import findspark
import pyspark
from pyspark.sql import SparkSession

# Faz a Interafce entre o Spark e o Jupyter Notebook
findspark.init()

# Inicializando uma Sessão no Spark
spark = SparkSession.builder.appName("Pepilines_tune").getOrCreate()

23/03/05 23:00:34 WARN Utils: Your hostname, 01bits resolves to a loopback address: 127.0.1.1; using 192.168.0.109 instead (on interface wlp6s0)
23/03/05 23:00:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/05 23:00:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


---

<center> <h1 style="background-color:seagreen; color:white" >Construindo Pipelines com Spark com Multi Layers Perceptron usando o DataSet Íris</h1></center>




<center> <h2 style="background-color:DarkKhaki; color:white" >Carregando o Cunjunto de Dados Íris</h2></center> 

In [2]:
iris = spark.read.csv("../Material_do_Curso/iris.csv", header=True, inferSchema=True, sep=",")
iris.show(5)

+-----------+----------+-----------+----------+-----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|
+-----------+----------+-----------+----------+-----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|
+-----------+----------+-----------+----------+-----------+
only showing top 5 rows



<center> <h2 style="background-color:DarkKhaki; color:white" >Separando os dados entre conjunto de traino e teste</h2></center> 

In [3]:
iris_train, iris_test = iris.randomSplit([0.8, 0.2])


<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Colocando os dados no formato aceito pelo Spark</h3></center>

In [4]:
from pyspark.ml.feature import VectorAssembler

In [5]:
vector = VectorAssembler(inputCols=["sepallength", "sepalwidth",
                                    "petallength", "petalwidth"], outputCol="independente")


<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Convertendo um tipo categórico em numérico com StringIndexer</center>

In [6]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="class", outputCol="label")

---

<center> <h2 style="background-color:DarkKhaki; color:white" >Importação do Módulo do PySpark Para Criação do Modelo de Rde Neural Artificial</h2></center>

In [7]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

mlp = MultilayerPerceptronClassifier(maxIter=100, layers=[4, 5, 4, 3],
                                     featuresCol="independente", labelCol="label")

---

<center> <h1 style="background-color:seagreen; color:white" >Criando o Pipeline e Aplicando Train-Validation Split</h1></center> 

 #### Train-Validation Split

> + Testa cada configuração do grid de hiper parâmetros uma única vez 
> + trainingratio define a proporção usada para treino
> + Quando encontrar o melhor conjunto de hiper parâmetros, ele novamente cria o modelo usando todos os dados de treino


<br>



In [8]:
from pyspark.ml import Pipeline

In [9]:
pipeline = Pipeline(stages=[vector, indexer, mlp])


<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Aplicando o Train-Validation Split</center>

In [10]:
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [11]:
performace = MulticlassClassificationEvaluator(labelCol="label", metricName="accuracy")
grid = ParamGridBuilder().addGrid(mlp.maxIter, [10, 100, 1000])
grid = grid.addGrid(mlp.layers, [[4, 5, 4, 3], [4, 4, 3]])
grid = grid.build()

train_val_split = TrainValidationSplit(estimator=pipeline,
                                       estimatorParamMaps=grid,
                                       evaluator=performace,
                                       trainRatio=0.8)

<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Executando o Pipiline com o Tune do Modelo</center>

In [12]:
modelo = train_val_split.fit(iris_train)

23/03/05 23:05:57 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.5
23/03/05 23:05:59 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.5
23/03/05 23:05:59 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.5
23/03/05 23:06:00 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.25
23/03/05 23:06:00 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.125
23/03/05 23:06:00 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.0625
23/03/05 23:06:00 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.03125
23/03/05 23:06:00 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.015625
23/03/05 

<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Teste do Modelo usando o conjunto teste</center>

In [13]:
previsao = modelo.transform(iris_test)

In [14]:
previsao.select("independente", "label",
                "prediction", "rawprediction",
                "probability").show(10)

+-----------------+-----+----------+--------------------+--------------------+
|     independente|label|prediction|       rawprediction|         probability|
+-----------------+-----+----------+--------------------+--------------------+
|[4.4,3.0,1.3,0.2]|  0.0|       0.0|[392.835663012028...|[1.0,9.9888295566...|
|[4.8,3.4,1.6,0.2]|  0.0|       0.0|[392.835758371839...|[1.0,9.9877640959...|
|[5.0,2.0,3.5,1.0]|  1.0|       1.0|[-214.59208852910...|[1.32631399252216...|
|[5.0,3.2,1.2,0.2]|  0.0|       0.0|[392.835754889314...|[1.0,9.9878030044...|
|[5.0,3.3,1.4,0.2]|  0.0|       0.0|[392.835761894589...|[1.0,9.9877247382...|
|[5.0,3.4,1.6,0.4]|  0.0|       0.0|[392.835768349100...|[1.0,9.9876526260...|
|[5.0,3.5,1.6,0.6]|  0.0|       0.0|[392.835771581009...|[1.0,9.9876165181...|
|[5.6,3.0,4.1,1.3]|  1.0|       1.0|[-28.235279879965...|[6.73848717495993...|
|[5.8,2.7,5.1,1.9]|  2.0|       2.0|[-255.25856942550...|[8.55699068064350...|
|[5.8,2.8,5.1,2.4]|  2.0|       2.0|[-255.2585691517


---

<center> <h1 style="background-color:seagreen; color:white" >Avaliação do Modelo usando PySpark</h1></center> 
<br>


In [15]:
performace = MulticlassClassificationEvaluator(labelCol="label",
                                               predictionCol="prediction",
                                               metricName="accuracy")
acuracia = performace.evaluate(previsao)
print(f"Acurácia do Modelo: {acuracia*100}%")

Acurácia do Modelo: 100.0%
